In [36]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import requests
import pandas as pd

In [52]:
def job_offers_wtj(
        job_title: str = "data analyst",
        pages: int = 1
):
    """
    ---
    Web scraping WTJ
    ---
    Lance un navigateur et créée un dataframe contenant les informations de
    chaque offre d'emploi pour un nombre de pages définis sur le site
    Welcome To The Jungle.
    ---
    Paramètres :
    - job_title: str : Le nom du métier pour lequel rechercher des offres.
    - pages: int : Le nombre de pages sur lesquels rechercher les offres.
    ---
    Retourne :
    - Le df contenant les informations de toute les offres récupérées.
    """
    # Instanciation de la liste contenant les liens pour les requêtes APIs.
    api_links = []
    # Lien de l'API de Welcome To The Jungle pour récupérer les données.
    api_link = f"https://api.welcometothejungle.com/api/v1/organizations"
    job = job_title.lower().replace(" ", "+")
    # Instanciation du driver Firefox.
    driver = webdriver.Firefox()
    # Instanciation du dataframe final.
    full_df = pd.DataFrame()
    # Nom des colonnes à garder dans le dataframe final.
    cols_to_keep = [
        "name",
        "salary_period",
        "experience_level",
        "apply_url",
        "contract_duration_min",
        "office.city",
        "office.address",
        "office.district",
        "office.latitude",
        "office.longitude",
        "office.zip_code",
        "profession.category.fr",
        "profession.name.fr"
        "name",
        "education_level",
        "application_fields.mode",
        "application_fields.name",
        "description",
        "organization.average_age",
        "organization.creation_year",
        "organization.default_language",
        "organization.description",
        "organization.industry",
        "organization.nb_employee",
        "contract_type",
        "salary_min",
        "salary_max",
        "education_level",
        "remote"
    ]
    try:
        for i in range(1, pages+1):
            url = f"https://www.welcometothejungle.com/fr/jobs?refinementList%5Boffices.country_code%5D%5B%5D=FR&query={job}&page={i}"
            # Ouvre chaque page sur le navigateur.
            driver.get(url)
            try:
                # Récupère le lien de chaque offre d'emploi sur la page.
                contents = WebDriverWait(driver, 20).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".sc-6i2fyx-0.gIvJqh"))
                )
                for content in contents:
                    link = content.get_attribute("href")
                    end_link = re.findall(r"/companies(.+)", link)[0]
                    full_link = api_link + end_link
                    # Rajoute le lien de chaque offre à la liste.
                    api_links.append(full_link)
            except Exception as e:
                print(f"Error scraping page {i} : {e}")
    finally:
        driver.quit()
    # Pour chaque lien de la liste, fait une requête API et stocke les informations dans un dataframe.
    for link_ in api_links:
        r = requests.get(link_)
        df = pd.json_normalize(
            r.json()["job"]
        )
        full_df = pd.concat([full_df, df], ignore_index=True)
    # Instanciation de la liste des colonnes à drop.
    cols_to_drop = [col for col in full_df.columns if col not in cols_to_keep]
    df = full_df.drop(columns=cols_to_drop)
    return df

In [53]:
df = job_offers_wtj("data analyst", 1)

In [28]:
from bs4 import BeautifulSoup
import pandas as pd
def clean_html(text):
    soup = BeautifulSoup(text, "html.parser")
    cleaned_text = soup.get_text(separator=" ")
    cleaned_text = cleaned_text.replace("\xa0", " ")
    return cleaned_text

df["description"] = df["description"].apply(clean_html)
df["organization.description"] = df["organization.description"].apply(clean_html)

In [47]:
api_links

['https://api.welcometothejungle.com/api/v1/organizations/margo/jobs/margo-analytics-data-engineer-h-f_paris?q=b7df709151a656b71b49ca574a776b75&o=579036&p=true',
 'https://api.welcometothejungle.com/api/v1/organizations/wttj/jobs/team-lead-data-engineering_paris?q=b7df709151a656b71b49ca574a776b75&o=1803168&p=true',
 'https://api.welcometothejungle.com/api/v1/organizations/matera/jobs/data-analyst-stage-de-6-mois_paris?q=ea94e9f6f4fa93da7f238e35ed2f633c&o=2228430',
 'https://api.welcometothejungle.com/api/v1/organizations/ermeo/jobs/data-analyst-stage-de-fin-d-etude_paris?q=ea94e9f6f4fa93da7f238e35ed2f633c&o=1979955',
 'https://api.welcometothejungle.com/api/v1/organizations/vente-unique/jobs/data-analyst-h-f_paris_VENTE_Z8zwM1o?q=ea94e9f6f4fa93da7f238e35ed2f633c&o=2293214',
 'https://api.welcometothejungle.com/api/v1/organizations/stockly/jobs/data-analyst-internship-paris_paris_STOCK_mzKkzAJ?q=ea94e9f6f4fa93da7f238e35ed2f633c&o=2269637',
 'https://api.welcometothejungle.com/api/v1/org

In [26]:
df.iloc[0]["description"]

"Margo Analytics est l'entité experte  de Margo Group des problématiques  Data, Cloud et DevOps  créée en 2020 par leurs fondateurs Raphaël et Mounir. Aujourd’hui  60 consultants  ont intégré l'entité et nous avons commencé à travailler avec  18 nouveaux clients  (Banque, Industrie, Assurance, Énergie, E commerce, Santé). A leurs côtés, vous pourrez évoluer rapidement et développer de nouvelles compétences.  Deux ADN fondateurs forts  et spécifiques à Margo Analytics à l’origine de l’entité : -  Toujours se positionner sur  les plus beaux sujets  et sur les  missions à fortes valeurs ajoutées -  Recruter  des  consultants passionnés  et  curieux  qui cherchent à être  challengés  Aujourd’hui, Margo Analytics possède  4 communautés  de compétences :  - Data engineer   - Data Science/ IA  - Galaxy OPS (devOps, dataOps, cloudOps) - Architecte Big Data  Ainsi en rejoignant Margo Analytics vous aurez le choix des missions (#consultantfirst) sur lesquelles vous souhaitez travailler. Vous ser

In [29]:
df.head()

,salary_max,apply_url,description,contract_type,contract_duration_min,salary_min,education_level,experience_level,salary_period,remote,profession.category.fr,office.address,office.city,office.district,office.latitude,office.longitude,office.zip_code,organization.average_age,organization.creation_year,organization.default_language,organization.description,organization.industry
0,None,https://jobs.lever.co/margo-group/dfd990e5-a46...,Margo Analytics est l'entité experte de Margo...,FULL_TIME,None,None,BAC_5,None,none,partial,Tech,"1, Rue de Saint-Pétersbourg, Paris",Paris,Paris,48.879700,2.323810,75008,30,2005,fr,"Plus qu’un simple groupe de conseil IT, MARGO ...","Software, IT / Digital"
1,None,None,The Data team at Welcome to the Jungle: Part o...,FULL_TIME,None,None,None,None,none,fulltime,Tech,"24, Rue du Mail, Paris",Paris,Paris,48.866950,2.343140,75002,32,2015,en,Welcome to the Jungle’s mission is simple: to ...,"Media, Recruitment"
2,1200,https://matera.recruitee.com/o/data-analyst-st...,Afin de mieux comprendre nos clients et leurs ...,INTERNSHIP,5,1000,BAC_5,LESS_THAN_6_MONTHS,none,punctual,Tech,"10, Rue Treilhard, Paris",Paris,Paris,48.877120,2.314520,75008,30,2017,fr,"Matera réinvente notre rapport au logement, en...","SaaS / Cloud Services, Commercial Real Estate,..."
3,None,None,As Data Analyst you will perform in-depth data...,INTERNSHIP,6,None,BAC_5,LESS_THAN_6_MONTHS,none,partial,Tech,157 boulevard macdonald,Paris,Paris,48.898163,2.378094,75019,28,2015,en,Causeway Technologies SAS based in Paris provi...,"Mobile Apps, Organization / Management, SaaS /..."
4,None,https://careers.flatchr.io/vacancy/jaonxpv16ma...,Descriptif du poste : Nous recherchons notre...,FULL_TIME,None,None,BAC_5,2_TO_3_YEARS,none,punctual,Tech,None,Paris,Paris,48.883950,2.385630,75019,31,2006,fr,"🚀 Pure player visionnaire, Vente-unique a été ...","Lifestyle, E-commerce"
